In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Dropout
import tushare as ts

ts.set_token('567d7e8ae547e5cc470120f8931ac62104b42d4256ccfb7770d983d4')
pro = ts.pro_api()
result = pd.DataFrame(columns=['percent','name','ts_code'])
stocks = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name,area,industry')

ts.set_token('b0030b7297c6b297f1db549c7f8c8a080b5f28e2267f07c4741e69ad')
pro = ts.pro_api()
print(stocks)
for stock in stocks.values:
    data = pro.daily(ts_code=stock[0], start_date='20110630', end_date='20230703')
    data = data[['close', 'vol']]
    data.sort_index(ascending=False, inplace=True)
    data = data.reset_index(drop=True)

    # 计算差值
    data_diff = data.diff().dropna()

    # 数据缩放
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_diff)

    lookback = 30

    # 定义生成器函数
    def generator(data, lookback=10, delay=1, batch_size=10):
        max_index = len(data) - delay - 1
        i = lookback
        while 1:
            if i + delay >= max_index:
                i = lookback
            samples = np.zeros((batch_size, lookback, data.shape[-1]))
            targets = np.zeros((batch_size,))  # targets should be 1D, single value for next prediction
            for j in range(batch_size):
                if i + delay >= max_index:
                    i = lookback
                rows = np.arange(i - lookback, i)
                samples[j] = data[rows]
                targets[j] = data[i + delay][0]  # target is the next value after the sequence
                i += 1
            yield samples, targets

    # 定义模型
    inputs = Input(shape=(lookback, data.shape[-1]))
    x = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
    x = Dropout(0.5)(x)
    x = LSTM(200, return_sequences=False)(x)  # add an additional LSTM layer
    outputs = Dense(1)(x)  # only predict the next value
    model = Model(inputs=inputs, outputs=outputs)


    model.compile(optimizer=Adam(), loss='mse')

    #...初始化代码保持不变
    train_data = data_scaled[:int(len(data_scaled)*0.9)]
    val_data = list(data_scaled[int(len(data_scaled)*0.1):])

    #预测涨跌
    correct_predictions = 0
    total_predictions = 0

    for epoch in range(30):  # train for 30 epochs
        train_gen = generator(train_data, lookback=lookback)
        model.fit(train_gen, epochs=1, steps_per_epoch=20, verbose=0)  # train the model for one epoch

        # move the first data point from val_data to train_data
        train_data = np.concatenate([train_data, np.reshape(val_data.pop(0), (1, -1))])

        # 预测下一个点
        last_sequence = np.reshape(train_data[-lookback:], (1, lookback, data_diff.shape[-1]))
        next_prediction = model.predict(last_sequence, verbose=0)[0, 0]

        # 计算预测值与真实值之间的差距
        if len(val_data) > 0:
            next_real = val_data[0][0]  # the next real value
            # error = np.abs((next_prediction - next_real) / next_real)  # the relative error
            # print(f'Epoch {epoch + 1}, validation error: {error:.4f}')

            # 对预测结果进行涨跌预测
            prediction_increase = next_prediction > 0
            real_increase = next_real > 0
            if prediction_increase == real_increase:
                correct_predictions += 1
            total_predictions += 1

    # 计算预测准确率
    accuracy = correct_predictions / total_predictions
    print(f'Prediction accuracy: {accuracy:.4f}')
    new_result = pd.DataFrame([[accuracy,stock[0],stock[1]]], columns=result.columns)
    print(new_result)
    result.append(new_result)
    new_result.to_csv('result.csv', mode='a', header=False, index=False)
    



        ts_code     name area industry
0     000001.SZ     平安银行   深圳       银行
1     000002.SZ      万科A   深圳     全国地产
2     000004.SZ     国华网安   深圳     软件服务
3     000005.SZ     ST星源   深圳     环境保护
4     000006.SZ     深振业A   深圳     区域地产
...         ...      ...  ...      ...
5222  873305.BJ     九菱科技   湖北     机械基件
5223  873339.BJ     恒太照明   江苏      元器件
5224  873527.BJ      夜光明   浙江     化工原料
5225  873593.BJ     鼎智科技   江苏     机械基件
5226  689009.SH  九号公司-WD   北京      摩托车

[5227 rows x 4 columns]


2023-07-04 16:06:31.600590: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1, validation error: 0.7985
Epoch 2, validation error: 1.0633


2023-07-04 16:06:33.421168: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-04 16:06:33.612936: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 3, validation error: 1.3387
Epoch 4, validation error: 0.8179


...

Epoch 21, validation error: 0.0748
Epoch 22, validation error: 1.0687


2023-07-04 16:08:52.500277: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-04 16:08:52.697181: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 23, validation error: 0.8499


2023-07-04 16:08:52.891150: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


KeyboardInterrupt: 

In [32]:
future = 1
data_to_predict = data_scaled[-lookback:]  # 最后lookback个数据点
predicted_data = []
for _ in range(future):
    samples = np.reshape(data_to_predict[-lookback:], (1, lookback, data_diff.shape[-1]))
    prediction = model.predict(samples,verbose=0)
    # 创建一个与输入数据具有相同特征数量的预测扩展
    prediction_extended = np.zeros(data_diff.shape[-1])
    prediction_extended[0] = prediction[0]  # assign predicted value to first feature
    predicted_data.append(prediction_extended)  # add prediction to list
    # 使用模型的最后一个预测值和前面的实际值（除去最早的一个）来形成新的输入窗口
    data_to_predict = np.vstack([data_to_predict[1:], prediction_extended])  

# 将差值转化为原始预测值
predicted_data_array = np.array(predicted_data)  # convert list to array
predicted_diffs = scaler.inverse_transform(predicted_data_array)
predicted_data = np.cumsum(predicted_diffs, axis=0) + data.iloc[-lookback-1: -lookback].values

print(round((predicted_data[-1][0]-data.values[-1][0])/data.values[-1][0]*100,2))


-11.94
